# **1. Dataset de Características**

In [ ]:
import cv2
import glob
import os
import numpy as np
import tensorflow as tf

In [ ]:
path='/kaggle/input/face-expression-recognition-dataset/images/validation/'

In [ ]:
folders=os.listdir(path)

In [ ]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential

In [ ]:
base_model=tf.keras.applications.resnet.ResNet101(
            include_top=False,
            weights='imagenet',
            input_shape=(224,224,3)
)

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Flatten())

In [ ]:
def get_convfeatures(path):
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(224,224))
    return model.predict(np.array([img]))

In [ ]:
features=[]
names=[]

In [ ]:
path="/kaggle/input/landscape-pictures/"
folders=['']

In [ ]:
for folder in folders:
    for fileName in glob.glob(path+folder+'/*.jpg'):
        names.append(fileName)
        features.append( get_convfeatures(fileName) )

In [ ]:
features=np.array(features)
names=np.array(names) 

In [ ]:
features = features[:,0,:]

# **2. K-means Clustering**

In [ ]:
from nltk.cluster.kmeans import KMeansClusterer
from nltk.cluster.util import cosine_distance

In [ ]:
kmeans=KMeansClusterer(7, distance=cosine_distance)

In [ ]:
cluster = kmeans.cluster(features,assign_clusters=True)

In [ ]:
cluster

In [ ]:
kmeans.means()

In [ ]:
group_images=[]
for centroid in kmeans.means():
    dist=[cosine_distance(centroid,feature) for feature in features]
    dist=np.array(dist)
    min_index=dist.argsort()[:5]
    group_images.append(names[min_index])

In [ ]:
group_images

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"]=(10,10)
_,axarr=plt.subplots(len(group_images),len(group_images[0]))
for i, cluster in enumerate(group_images):
    for j,path_image in enumerate(cluster):
        img=cv2.imread(path_image)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        axarr[i,j].imshow(img)